In [23]:
from tqdm import tqdm_notebook
import os
import sys
import urllib.request
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
from selenium.common.exceptions import NoSuchElementException

In [93]:
df = pd.read_csv('./project_data/(O)data_phy.facility(finished).csv', encoding='utf-8')
df

,Unnamed: 0,names,site.area,parking.area
0,5,동작테니스장,7868.0,NaN
1,7,독산동잔디축구장,18005.0,NaN
2,8,강동그린웨이가족캠핑장,125021.0,40.0
3,9,한강시민공원광나루지구,6000.0,404.0
4,10,구의야구공원 야구장,49270.0,22.0
5,11,한강시민공원뚝섬지구,NaN,1000.0
6,13,우장산공원,10200.0,NaN
7,38,중랑구립잔디운동장,13304.0,NaN
8,40,한강시민공원난지지구,NaN,556.0
9,41,한강시민공원망원지구,NaN,613.0


In [94]:
name_list=df[df.columns[1]]

In [95]:
name_list

0               동작테니스장
1             독산동잔디축구장
2          강동그린웨이가족캠핑장
3          한강시민공원광나루지구
4           구의야구공원 야구장
5           한강시민공원뚝섬지구
6                우장산공원
7            중랑구립잔디운동장
8           한강시민공원난지지구
9           한강시민공원망원지구
10          한강시민공원잠원지구
11          한강시민공원잠실지구
12              신월야구공원
13          한강시민공원양화지구
14          한강시민공원이촌지구
15               잠실야구장
16               잠실체육관
17              잠실1수영장
18            잠실인라인연습장
19             잠실인라인트랙
20            잠실인라인하키장
21             잠실풋살경기장
22               목동야구장
23            목동다목적경기장
24               효창운동장
25            창동문화체육센터
26             동대문구체육관
27             관악구민운동장
28            관악구민체육센터
29           방배배수지체육공원
30            은평구민체육센터
31              어린이대공원
32         서대문구립인조잔디구장
33             올림픽주경기장
34             잠실보조경기장
35             은평구립축구장
36                 황학정
37              용마폭포공원
38           노해근린공원운동장
39              마들스타디움
40            서울교대체육시설
41           양재근린공원축구장
42              양재체육시설
43         

In [123]:
han_river_list = ['서울색공원','여의도 한강공원', '선유도공원','반포 한강공원', '뚝섬 한강공원', '잠원 한강공원', '난지 한강공원', '잠실 한강공원', '망원 한강공원', '이촌 한강공원', '양화 한강공원', '여의도샛강생태공원','광나루 한강공원','강서 한강공원','세빛섬한강공원','서울색공원','암사둔치생태공원',]

In [124]:
han_river_list

['서울색공원',
 '여의도 한강공원',
 '선유도공원',
 '반포 한강공원',
 '뚝섬 한강공원',
 '잠원 한강공원',
 '난지 한강공원',
 '잠실 한강공원',
 '망원 한강공원',
 '이촌 한강공원',
 '양화 한강공원',
 '여의도샛강생태공원',
 '광나루 한강공원',
 '강서 한강공원',
 '세빛섬한강공원',
 '서울색공원',
 '암사둔치생태공원']

In [126]:
driver = webdriver.Chrome('./chromedriver.exe')
url_page = 'https://map.naver.com/'
addr_list = []
error_index_list = []
i=-1

for name in tqdm_notebook(name_list):
    
    i+=1
    driver.get(url_page)
    try:
        element = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "search-input")))
    except Exception as e:
        print('오류발생1',e)
    driver.find_element_by_id('search-input').send_keys('서울 '+ str(name))
    try:
        element = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "fieldset")))
    except Exception as e:
        print('오류발생2',e)
    driver.find_element_by_css_selector('fieldset > button').click()
    driver.implicitly_wait(5)
    try:
        addr = driver.find_element_by_css_selector('dl.lsnx_det > dd.addr').text
    except :
        try : addr = driver.find_element_by_css_selector('div.lanx_t > a').text
        except NoSuchElementException as e:
            error_index_list.append(i)           
            continue
    addr_list.append(addr)
    addr_list

driver.quit()

In [127]:
df.loc[error_index_list]

,Unnamed: 0,전철역명,X,Y


In [98]:
df = df.drop(error_index_list).reset_index(drop=True)

In [99]:
df

,Unnamed: 0,names,site.area,parking.area
0,5,동작테니스장,7868.0,NaN
1,7,독산동잔디축구장,18005.0,NaN
2,8,강동그린웨이가족캠핑장,125021.0,40.0
3,9,한강시민공원광나루지구,6000.0,404.0
4,10,구의야구공원 야구장,49270.0,22.0
5,11,한강시민공원뚝섬지구,NaN,1000.0
6,13,우장산공원,10200.0,NaN
7,38,중랑구립잔디운동장,13304.0,NaN
8,40,한강시민공원난지지구,NaN,556.0
9,41,한강시민공원망원지구,NaN,613.0


In [100]:
len(addr_list)

51

In [128]:
addr_list_new=[]
for word in addr_list:
    print(word)
    addr_list_new.append(word.replace(' 지번','')) # 지번 단어 없애기

서울특별시 영등포구 여의동로 330 지번
서울특별시 영등포구 여의동로 330 한강사업본부 여의도안내센터 지번
서울특별시 영등포구 선유로 343 지번
서울특별시 서초구 신반포로11길 40 한강공원 반포 안내센터 지번
서울특별시 광진구 강변북로 139 한강사업본부뚝섬안내 지번
서울특별시 서초구 잠원동 잠원로 26-10
서울특별시 마포구 한강난지로 162 한강공원 난지안내센터 지번
서울특별시 송파구 한가람로 65 한강사업본부 잠실안내센터 지번
서울특별시 마포구 마포나루길 467 한강공원망원지구사무소 지번
서울특별시 용산구 이촌로72길 62 한강공원 이촌안내센터 지번
서울특별시 영등포구 당산동6가 98-1
서울특별시 영등포구 여의도동
서울특별시 강동구 선사로 83-66 한강사업본부 광나루안내센터 지번
서울특별시 강서구 방화동 2-32
서울특별시 서초구 반포동
서울특별시 영등포구 여의동로 330 지번
서울특별시 강동구 암사동 616-1


In [129]:
addr_list_new

['서울특별시 영등포구 여의동로 330',
 '서울특별시 영등포구 여의동로 330 한강사업본부 여의도안내센터',
 '서울특별시 영등포구 선유로 343',
 '서울특별시 서초구 신반포로11길 40 한강공원 반포 안내센터',
 '서울특별시 광진구 강변북로 139 한강사업본부뚝섬안내',
 '서울특별시 서초구 잠원동 잠원로 26-10',
 '서울특별시 마포구 한강난지로 162 한강공원 난지안내센터',
 '서울특별시 송파구 한가람로 65 한강사업본부 잠실안내센터',
 '서울특별시 마포구 마포나루길 467 한강공원망원지구사무소',
 '서울특별시 용산구 이촌로72길 62 한강공원 이촌안내센터',
 '서울특별시 영등포구 당산동6가 98-1',
 '서울특별시 영등포구 여의도동',
 '서울특별시 강동구 선사로 83-66 한강사업본부 광나루안내센터',
 '서울특별시 강서구 방화동 2-32',
 '서울특별시 서초구 반포동',
 '서울특별시 영등포구 여의동로 330',
 '서울특별시 강동구 암사동 616-1']

In [132]:
client_id = "mBvaixbFoTsoinOJvYyX"
client_secret = "RsgWab9vxl"
lat_lon_dic = {}
lat_lon_list = []
i = 0
error_index = []

for addr in tqdm_notebook(addr_list_new):   
    try:
        encText = urllib.parse.quote(addr)
        url = "https://openapi.naver.com/v1/map/geocode?query=" + encText # json 결과
        # url = "https://openapi.naver.com/v1/map/geocode.xml?query=" + encText # xml 결과
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()

        if(rescode==200):
            response_body = response.read()
            json_data = response_body.decode('utf-8') # 리턴값이 json
            location = json.JSONDecoder().decode(json_data) # json 파일 디코드
            lat_lon_dic['lon'] = location['result']['items'][0]['point']['x']
            lat_lon_dic['lat'] = location['result']['items'][0]['point']['y']

    #         print(num, rescode, addr,lat_lon_dic['lon'],  lat_lon_dic['lat'])
            lat_lon_list.append(lat_lon_dic.copy()) # copy() immutable 하게 바꾸기
            print(addr)
        else:
            print("Error Code:" + rescode)
    
    except Exception as e :
        print('Not Found' + addr + '\t' + str(i))
        error_index.append(i)
    i = i+1

서울특별시 영등포구 여의동로 330
서울특별시 영등포구 여의동로 330 한강사업본부 여의도안내센터
서울특별시 영등포구 선유로 343
서울특별시 서초구 신반포로11길 40 한강공원 반포 안내센터
서울특별시 광진구 강변북로 139 한강사업본부뚝섬안내
Not Found서울특별시 서초구 잠원동 잠원로 26-10	5
서울특별시 마포구 한강난지로 162 한강공원 난지안내센터
서울특별시 송파구 한가람로 65 한강사업본부 잠실안내센터
서울특별시 마포구 마포나루길 467 한강공원망원지구사무소
서울특별시 용산구 이촌로72길 62 한강공원 이촌안내센터
Not Found서울특별시 영등포구 당산동6가 98-1	10
서울특별시 영등포구 여의도동
서울특별시 강동구 선사로 83-66 한강사업본부 광나루안내센터
서울특별시 강서구 방화동 2-32
서울특별시 서초구 반포동
서울특별시 영등포구 여의동로 330
서울특별시 강동구 암사동 616-1


In [135]:
lat_lon_list.append({'lon':126.902274, 'lat': 37.538425})

In [136]:
loc_df = pd.DataFrame(lat_lon_list)

In [137]:
loc_df

,lat,lon
0,37.526337,126.933597
1,37.526337,126.933597
2,37.542360,126.901817
3,37.507687,126.992702
4,37.529346,127.074006
5,37.562839,126.885441
6,37.517630,127.086763
7,37.552343,126.899887
8,37.517835,126.970798
9,37.528317,126.929425


In [107]:
error_index # error_index는 해당 로우를 삭제해야함

[6, 10, 13]

In [108]:
name_df = pd.DataFrame(name_list)

In [109]:
name_df

,names
0,동작테니스장
1,독산동잔디축구장
2,강동그린웨이가족캠핑장
3,한강시민공원광나루지구
4,구의야구공원 야구장
5,한강시민공원뚝섬지구
6,우장산공원
7,중랑구립잔디운동장
8,한강시민공원난지지구
9,한강시민공원망원지구


In [110]:
name_df = name_df.drop(error_index_list).reset_index(drop=True) 

In [111]:
name_df = name_df.drop(error_index).reset_index(drop=True) 
# 해당 로우 삭제하고 리셋 인덱스, drop=True : 기존 인덱스 생성 x

In [112]:
name_df.head(81)

,names
0,동작테니스장
1,독산동잔디축구장
2,강동그린웨이가족캠핑장
3,한강시민공원광나루지구
4,구의야구공원 야구장
5,한강시민공원뚝섬지구
6,중랑구립잔디운동장
7,한강시민공원난지지구
8,한강시민공원망원지구
9,한강시민공원잠실지구


In [113]:
gu = []
for full_address in addr_list_new: # 구 별로 데이터 자르기
    addr_split = full_address.split()
    gu.append(addr_split[1])
gu

gu_df = pd.DataFrame(gu, columns=['구'])
gu_df = gu_df.drop(error_index).reset_index(drop=True) 
gu_df

,구
0,동작구
1,금천구
2,강동구
3,강동구
4,광진구
5,광진구
6,중랑구
7,마포구
8,마포구
9,송파구


In [114]:
name_final = pd.concat([name_df,loc_df,gu_df], axis=1)

In [115]:
name_final.columns

Index(['names', 'lat', 'lon', '구'], dtype='object')

In [116]:
name_final = name_final[[name_final.columns[0],name_final.columns[3],'lat','lon']]

In [119]:
name_final.head(100)

,names,구,lat,lon
0,동작테니스장,동작구,37.499200,126.973850
1,독산동잔디축구장,금천구,37.470260,126.896938
2,강동그린웨이가족캠핑장,강동구,37.536107,127.153307
3,한강시민공원광나루지구,강동구,37.555461,127.122583
4,구의야구공원 야구장,광진구,37.547229,127.096827
5,한강시민공원뚝섬지구,광진구,37.529346,127.074006
6,중랑구립잔디운동장,중랑구,37.606910,127.113001
7,한강시민공원난지지구,마포구,37.562839,126.885441
8,한강시민공원망원지구,마포구,37.552343,126.899887
9,한강시민공원잠실지구,송파구,37.517630,127.086763


In [138]:
loc_df.to_csv('./project_data/hanriver_park.csv',sep=',', encoding='utf-8')